In [1]:
import pandas as pd
import os
import pickle
from utils import * 

In [2]:
def from_csv_gt_to_dict(gt_path, gt_file,benchmark_name):
    gt = pd.read_csv(os.path.join(gt_path, gt_file))
    join_dict = {}
    for index, row in gt.iterrows():
        query_pair = (row['query_table'], row['query_column'])
        candidate_pair = (row['candidate_table'], row['candidate_column'])
        if query_pair in join_dict.keys():
            join_dict[query_pair].append(candidate_pair)
        else:
            join_dict[query_pair] = [candidate_pair]
    with open(os.path.join(gt_path,f'join_dict_{benchmark_name}.pkl'), 'wb') as f:
        pickle.dump(join_dict, f)
    return join_dict

In [4]:
def join_benchmark_column_names_match_ratio(join_dict):
    ratios = []
    for key in join_dict.keys():
        ttl_candidates = len(join_dict[key])
        n_match_col = 0
        for candidate in join_dict[key]:
            if candidate[1] == key[1]:
                n_match_col += 1
        ratio = n_match_col/ttl_candidates
        ratios.append(ratio)

    final = sum(ratios)/len(ratios)
    return final

In [7]:
def avg_top_k_search(join_dict):
    avg = 0
    for key in join_dict.keys():
        avg += len(join_dict[key])
    return avg/len(join_dict.keys())

In [19]:
def is_query_in_candidate(join_dict):
    counter = 0
    for key in join_dict.keys():
        if key in join_dict[key]:
            counter += 1
    return counter

In [3]:
benchmark_name = 'webtable'
gt_path = f'/Users/alaaalmutawa/Documents/Thesis/lakebench/join/{benchmark_name}'
gt_file = f'{benchmark_name}_join_ground_truth.csv'
join_dict = from_csv_gt_to_dict(gt_path, gt_file,benchmark_name)

In [26]:
join_dict_path = ['/Users/alaaalmutawa/Documents/Thesis/lakebench/join/opendata/join_dict_opendata.pkl','/Users/alaaalmutawa/Documents/Thesis/lakebench/join/webtable/join_dict_webtable.pkl','/Users/alaaalmutawa/Documents/Thesis/lakebench/join/webtable/small/join_dict_webtable_small.pkl','/Users/alaaalmutawa/Documents/Thesis/lakebench/join/webtable/small_var2/join_dict_webtables_small_var2.pkl','/Users/alaaalmutawa/Documents/Thesis/nextiajd/testbedS/warpgate/join_dict_testbedS_warpgate.pkl','/Users/alaaalmutawa/Documents/Thesis/nextiajd/testbedS/join_dict_testbedS_warpgate_non-numerical_test.pkl','/Users/alaaalmutawa/Documents/Thesis/nextiajd/testbedS/10_candidates_non-numerical/join_dict_testbedS_10_non-numerical.pkl','/Users/alaaalmutawa/Documents/Thesis/nextiajd/testbedM/warpgate/join_dict_testbedM_warpgate.pkl'  ]
benchmark_names = ['opendata','webtable','webtables_small','webtables_small_var2','nextiajd_small','nextiajd_small_var2','nextiajd_small_var3','nextiajd_medium']
queries_count = []
alignment = []
ratio = []
avg_number_of_candidates = []
query_in_candidates = []

for i in range(len(join_dict_path)):
    with open(join_dict_path[i], 'rb') as f:
        join_dict = pickle.load(f)
    match_ratio = join_benchmark_column_names_match_ratio(join_dict)
    queries_count.append(len(join_dict))
    alignment.append(match_ratio)
    print(f'number of queries: {len(join_dict)} in benchmark {benchmark_names[i]}')
    print(f'ratio of candidate column names that align query column name: {match_ratio} in {benchmark_names[i]}')

number of queries: 128 in benchmark opendata
ratio of candidate column names that align query column name: 1.0 in opendata
number of queries: 2169 in benchmark webtable
ratio of candidate column names that align query column name: 1.0 in webtable
number of queries: 313 in benchmark webtables_small
ratio of candidate column names that align query column name: 1.0 in webtables_small
number of queries: 205 in benchmark webtables_small_var2
ratio of candidate column names that align query column name: 1.0 in webtables_small_var2
number of queries: 184 in benchmark nextiajd_small
ratio of candidate column names that align query column name: 0.3767899220344875 in nextiajd_small
number of queries: 177 in benchmark nextiajd_small_var2
ratio of candidate column names that align query column name: 0.3902787890076028 in nextiajd_small_var2
number of queries: 15 in benchmark nextiajd_small_var3
ratio of candidate column names that align query column name: 0.07314685314685314 in nextiajd_small_var3

In [27]:
for i in range(len(join_dict_path)):
    with open(join_dict_path[i], 'rb') as f:
        join_dict = pickle.load(f)
    avg = avg_top_k_search(join_dict)
    avg_number_of_candidates.append(avg)
    print(f'average number of candidates per query: {avg} in {benchmark_names[i]}')

average number of candidates per query: 8.890625 in opendata
average number of candidates per query: 15.907330567081605 in webtable
average number of candidates per query: 2.9680511182108624 in webtables_small
average number of candidates per query: 23.829268292682926 in webtables_small_var2
average number of candidates per query: 3.5706521739130435 in nextiajd_small
average number of candidates per query: 3.5480225988700567 in nextiajd_small_var2
average number of candidates per query: 12.466666666666667 in nextiajd_small_var3
average number of candidates per query: 4.37037037037037 in nextiajd_medium


In [28]:
for i in range(len(join_dict_path)):
    with open(join_dict_path[i], 'rb') as f:
        join_dict = pickle.load(f)
    counter = is_query_in_candidate(join_dict)
    if counter == len(join_dict.keys()):
        print(f'All queries are in the candidates {counter} out of {len(join_dict.keys())} ratio {counter/len(join_dict.keys())} in {benchmark_names[i]}')
    elif counter > 0:
        print(f'Some queries are in the candidates {counter} out of {len(join_dict.keys())} ratio {counter/len(join_dict.keys())} in {benchmark_names[i]}')
    else:
        print(f'No queries are in the candidates in {benchmark_names[i]}')
    query_in_candidates.append(counter)
    

Some queries are in the candidates 123 out of 128 ratio 0.9609375 in opendata
Some queries are in the candidates 1141 out of 2169 ratio 0.5260488704472107 in webtable
Some queries are in the candidates 149 out of 313 ratio 0.476038338658147 in webtables_small
Some queries are in the candidates 130 out of 205 ratio 0.6341463414634146 in webtables_small_var2
No queries are in the candidates in nextiajd_small
No queries are in the candidates in nextiajd_small_var2
No queries are in the candidates in nextiajd_small_var3
No queries are in the candidates in nextiajd_medium


In [30]:
summary_df = pd.DataFrame({'benchmark':benchmark_names,'queries_count':queries_count,'alignment':alignment,'avg_number_of_candidates':avg_number_of_candidates,'query_in_candidates':query_in_candidates})

In [33]:
summary_df.to_csv('join_benchmark_summary.csv',index=False)


In [34]:
summary_df

,benchmark,queries_count,alignment,avg_number_of_candidates,query_in_candidates
0,opendata,128,1.000000,8.890625,123
1,webtable,2169,1.000000,15.907331,1141
2,webtables_small,313,1.000000,2.968051,149
3,webtables_small_var2,205,1.000000,23.829268,130
4,nextiajd_small,184,0.376790,3.570652,0
5,nextiajd_small_var2,177,0.390279,3.548023,0
6,nextiajd_small_var3,15,0.073147,12.466667,0
7,nextiajd_medium,189,0.290572,4.370370,0


In [35]:
explaination = {
    'keyword':['benchmark','queries_count','alignment','avg_number_of_candidates','query_in_candidates'],
    'explaination':['The name of the benchmark','The number of queries in the benchmark','The ratio of column names matches that syntactically aligns with the query column name','The average number of matches per query in the benchmark','The number of queries that are in the candidates']
}
explaination_df = pd.DataFrame(explaination)

In [36]:
explaination_df

,keyword,explaination
0,benchmark,The name of the benchmark
1,queries_count,The number of queries in the benchmark
2,alignment,The ratio of column names matches that syntact...
3,avg_number_of_candidates,The average number of matches per query in the...
4,query_in_candidates,The number of queries that are in the candidates
